# Chapter 2. Why and When Should You Write Unit Tests?

## 2.1 Module outline

(1) What is unit testing for?

(2) Unit testing in your personal development process.

* Test first
* Test last
* Test driven

(3) Unit testing in the wider development process.

## 2.2 Four reasons for unit testing

What is unit testing for?

(1) Understand what to build

(2) Document the units

(3) Design the units

(4) Regression protection

## 2.3 Understand what to build

Collaborate with people in other roles to understand what's needed

(1) Business analyst, product owner

(2) Tester

(3) Interaction designer

(4) Lead developer

In [ ]:
# SAVE AS test_seat_finder.py

import unittest

from theatre import SeatFinder

# Note: A is the front row, so A6 is the 6th seat on the front row

class SeatFinderTest(unittest.TestCase):
    def test_prefer_near_the_front(self):
        finder = SeatFinder(available_seats={"A6", "B6", "C7"})
        seats = finder.find_seats(1)
        assert seats == {"A6"}

    def test_finds_adjacent_seats_when_more_than_one_requested(self):
        finder = SeatFinder(available_seats={"A6", "A8", "C6", "C7"})
        seats = finder.find_seats(2)
        assert seats == {"C6", "C7"}

    def test_finds_separate_seats_when_adjacent_not_available(self):
        finder = SeatFinder(available_seats={"A6", "B6", "C7"})
        seats = finder.find_seats(2)
        assert seats == {"B6", "A6"}

    def test_find_seats_fails_when_not_enough_available(self):
        finder = SeatFinder(available_seats={"A6", "B6", "C7"})
        seats = finder.find_seats(5)
        assert seats == {}

## 2.4 Documenting the units

"Executable specification"

* Tests document the behavior of the code
* How the unit is intended to be used

Example: add one more test case `test_find_seats_for_wheelchair_users_on_front_row`

In [ ]:
# SAVE AS test_seat_finder.py

import unittest

from theatre import SeatFinder

# Note: A is the front row, so A6 is the 6th seat on the front row

class SeatFinderTest(unittest.TestCase):
    def test_prefer_near_the_front(self):
        finder = SeatFinder(available_seats={"A6", "B6", "C7"})
        seats = finder.find_seats(1)
        assert seats == {"A6"}

    def test_finds_adjacent_seats_when_more_than_one_requested(self):
        finder = SeatFinder(available_seats={"A6", "A8", "C6", "C7"})
        seats = finder.find_seats(2)
        assert seats == {"C6", "C7"}

    def test_finds_separate_seats_when_adjacent_not_available(self):
        finder = SeatFinder(available_seats={"A6", "B6", "C7"})
        seats = finder.find_seats(2)
        assert seats == {"B6", "A6"}

    def test_find_seats_fails_when_not_enough_available(self):
        finder = SeatFinder(available_seats={"A6", "B6", "C7"})
        seats = finder.find_seats(5)
        assert seats == {}
        
    def test_find_seats_for_wheelchair_users_on_front_row(self):
        finder = SeatFinder(available_seats={"A1W", "A6", "A7", "C7"})
        seats = finder.find_seats(1, wheelchair_count=1)
        assert seats == {"A1W"}

## 2.5 Designing the units

(1) Decompose the problem into units that are independently testable.

* loose coupling

(2) Design the interface separately from doing the implementation.

In [ ]:
# SAVE AS telemetry.py

DiagnosticChannelConnectionString = "*111#"

class TelemetryDiagnosticControls:

    def __init__(self, telemetry_client=None):
        self.telemetry_client = telemetry_client or TelemetryClient()
        self.diagnostic_info = ""

    def check_transmission(self):
        telemetry_client = self.reconnect(DiagnosticChannelConnectionString)
        self.diagnostic_info = ""
        self.diagnostic_info = self.fetch_diagnostic_info(telemetry_client)

    def reconnect(self, address):
        self.telemetry_client.disconnect()
        retryLeft = 3
        while ((not self.telemetry_client.online_status) and retryLeft > 0):
            self.telemetry_client.connect(address)
            retryLeft -= 1

        if not self.telemetry_client.online_status:
            raise Exception("Unable to connect.")
        return self.telemetry_client

    def fetch_diagnostic_info(self, connected_client):
        connected_client.send(TelemetryClient.DIAGNOSTIC_MESSAGE)
        if not self.telemetry_client.online_status:
            raise Exception("Unable to connect.")
        return connected_client.receive()



class TelemetryClient(object):
    DIAGNOSTIC_MESSAGE = "AT#UD"

    def __init__(self):
        self.online_status = False
        self._diagnostic_message_result = ""

    def connect(self, telemetry_server_connection_string):
        if not telemetry_server_connection_string:
            raise Exception()

        # simulate the operation on a real modem
        success = random.randint(0, 10) <= 8
        self.online_status = success

    def disconnect(self):
        self.online_status = False

    def send(self, message):
        if not message:
            raise Exception()

        if message == TelemetryClient.DIAGNOSTIC_MESSAGE:
            # simulate a status report
            self._diagnostic_message_result = """\
LAST TX rate................ 100 MBPS\r\n
HIGHEST TX rate............. 100 MBPS\r\n
LAST RX rate................ 100 MBPS\r\n
HIGHEST RX rate............. 100 MBPS\r\n
BIT RATE.................... 100000000\r\n
WORD LEN.................... 16\r\n
WORD/FRAME.................. 511\r\n
BITS/FRAME.................. 8192\r\n
MODULATION TYPE............. PCM/FM\r\n
TX Digital Los.............. 0.75\r\n
RX Digital Los.............. 0.10\r\n
BEP Test.................... -5\r\n
Local Rtrn Count............ 00\r\n
Remote Rtrn Count........... 00"""

            return
        # here should go the real Send operation (not needed for this exercise)

    def receive(self):
        if not self._diagnostic_message_result:
            # simulate a received message (just for illustration - not needed for this exercise)
            message = ""
            messageLength = random.randint(0, 50) + 60
            i = messageLength
            while(i >= 0):
                message += chr((random.randint(0, 40) + 86))
                i -= 1
        else:
            message = self._diagnostic_message_result
            self._diagnostic_message_result = ""

        return message

## 2.6 Regression protection

Regression -- Something worked before and doesn't any more

* A unit test should fail & point out which unit failed and why

In [ ]:
# SAVE AS patient.py

class Patient:
    
    def __init__(self, prescriptions=None):
        self.prescriptions = prescriptions or []
    
    def add_prescription(self, prescription):
        self.prescriptions.append(prescription)
        
    def days_taking(self, medicine_name):
        prescriptions = filter(lambda p: p.name == medicine_name, self.prescriptions)
        days = set()
        for prescription in prescriptions:
            days.update(prescription.days_taken())
        return days
        
    def clash(self, medicine_names):
        days_taking = [self.days_taking(medicine_name) for medicine_name in medicine_names] or [set()]
        return set.intersection(*days_taking)

In [ ]:
# SAVE AS test_patient.py

from datetime import date, timedelta

from patient import Patient
from prescription import Prescription

def days_ago(days):
    return date.today() - timedelta(days=days)

class TestPatient:
    
    def test_clash_with_no_prescriptions(self):
        patient = Patient(prescriptions=[])
        assert patient.clash([]) == set()
        
    def test_clash_with_one_irrelevant_prescription(self):
        patient = Patient(prescriptions=[Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=2)])
        assert patient.clash(["Prozac"]) == set()
        
    def test_clash_with_one_prescription(self):
        patient = Patient(prescriptions=[Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=2)])
        assert patient.clash(["Codeine"]) == {days_ago(days=2), days_ago(days=1)}

    def test_clash_with_two_different_prescriptions(self):
        patient = Patient(prescriptions=[Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=2),
                                         Prescription("Prozac",     dispense_date = days_ago(days=2), days_supply=2)])
        assert patient.clash(["Codeine", "Prozac"]) == {days_ago(days=2), days_ago(days=1)}

    def test_clash_with_two_prescriptions_for_same_medication(self):
        patient = Patient(prescriptions=[Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=2),
                                         Prescription("Codeine", dispense_date = days_ago(days=3), days_supply=2)])
        assert patient.clash(["Codeine"]) == {days_ago(days=3), days_ago(days=2), days_ago(days=1)}

    def test_days_taking_for_irrelevant_prescription(self):
        patient = Patient(prescriptions=[Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=2)])
        assert patient.days_taking("Prozac") == set()

    def test_days_taking(self):
        patient = Patient(prescriptions=[Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=2),
                                         Prescription("Codeine", dispense_date = days_ago(days=3), days_supply=2)])
        assert patient.days_taking("Codeine") == {days_ago(days=3),
                                                     days_ago(days=2), 
                                                     days_ago(days=1)}
  
    def test_clash_overlapping_today(self):
        patient = Patient(prescriptions=[Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=3),
                                         Prescription("Prozac",     dispense_date = days_ago(days=2), days_supply=3)])
        assert patient.clash(["Codeine", "Prozac"]) == {days_ago(days=2), days_ago(days=1)}

In [ ]:
# SAVE AS prescription.py

from datetime import date, timedelta

class Prescription:
    
    def __init__(self, name, dispense_date, days_supply):
        self.name = name
        self.dispense_date = dispense_date
        self.days_supply = days_supply
        
    def days_taken(self):
        all_days = [self.dispense_date + timedelta(days=i) for i in range(self.days_supply)]
        # BUGBUG: The following change will cause a regression.
        return all_days #[day for day in all_days if day < date.today()]

In [ ]:
# SAVE AS test_prescription.py

from datetime import date, timedelta

from prescription import Prescription

def days_ago(days):
    return date.today() - timedelta(days=days)

class TestPrescription:
    
    def test_days_taken_excludes_future_dates(self):
        prescription = Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=4)
        assert prescription.days_taken() == [days_ago(2), days_ago(1)]

```bash
$ pip install pytest
$ python -m pytest
========================================================= test session starts =========================================================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/healthcare_example_unittest, inifile:
collected 9 items                                                                                                                     

test_patient.py .......F                                                                                                        [ 88%]
test_prescription.py F                                                                                                          [100%]

============================================================== FAILURES ===============================================================
______________________________________________ TestPatient.test_clash_overlapping_today _______________________________________________

self = <test_patient.TestPatient object at 0x7fd32eb5d860>

    def test_clash_overlapping_today(self):
        patient = Patient(prescriptions=[Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=3),
                                         Prescription("Prozac",     dispense_date = days_ago(days=2), days_supply=3)])
>       assert patient.clash(["Codeine", "Prozac"]) == {days_ago(days=2), days_ago(days=1)}
E       assert {datetime.dat...e(2018, 3, 7)} == {datetime.date...e(2018, 3, 6)}
E         Extra items in the left set:
E         datetime.date(2018, 3, 7)
E         Use -v to get the full diff

test_patient.py:47: AssertionError
_______________________________________ TestPrescription.test_days_taken_excludes_future_dates ________________________________________

self = <test_prescription.TestPrescription object at 0x7fd32eb54780>

    def test_days_taken_excludes_future_dates(self):
        prescription = Prescription("Codeine", dispense_date = days_ago(days=2), days_supply=4)
>       assert prescription.days_taken() == [days_ago(2), days_ago(1)]
E       assert [datetime.dat...e(2018, 3, 8)] == [datetime.date...e(2018, 3, 6)]
E         Left contains more items, first extra item: datetime.date(2018, 3, 7)
E         Use -v to get the full diff

test_prescription.py:12: AssertionError
================================================= 2 failed, 7 passed in 0.07 seconds ==================================================
```

## 2.7 Limitations of unit testing

### 2.7.1 Limitations of unit testing

(1) Testing can't find all the errors.

(2) Unit testing won't find integration errors.

### 2.7.2 Test first, test last, test driven

## 2.8 Testing as part of your personal development process

(1) Should develop a suite of automated tests for the production code

(2) Three extreme approaches:

* Test last
* Test first
* Test driven

## 2.9 Test last

Design code ==> Design tests ==> Debug & rework ==> Design code

(1) Advantages

* You don't invest in designing the automated test cases until your design is relatively stable.

(2) Risk

* Discover testability problems and bugs late in the process.

* You'll rush or skip designing the tests.

## 2.10 Test first

Design code ==> Design tests ==> Write code ==> Refactor code or refactor tests

(1) Advantages

* Force you to design a testable interface before you've invested much time in an implementation.

* Help you cover all the important features by the test cases.

(2) Risk:

Rework

## 2.11 Test driven

Write a test ==> Write a little code ==> Refactor ==> Write a test

This is the approach adopted by the author for developing examples in this course.

## 2.12 Continuous integration

Unit testing in the wider development process

(1) Version control

(2) Work from known good

(3) Unexpected failures

(4) Continuous integration

Catch unexpected failures by testing every change.

http://www.martinfowler.com/articles/continuousIntegration.html

(5) Coverage metrics

## 2.13 Module review

(1) What is unit testing for?

(2) Test first, test last, test driven

(3) Unit testing in the wider development process